In [1]:
%tensorflow_version 1.x #use tensorflow magic to use version 1.x in colab

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.x #use tensorflow magic to use version 1.x in colab`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [2]:
#imports
import pandas as pd
import numpy as np
import nltk
import io
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from keras.preprocessing import text as keras_text, sequence as keras_seq
from keras.utils import np_utils
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers import Dense, Activation, Convolution1D, MaxPooling1D, Dropout, Flatten
from keras.models import Sequential, save_model
from sklearn.model_selection import train_test_split
from keras.layers import Embedding
from sklearn.metrics import classification_report,confusion_matrix

Using TensorFlow backend.


In [3]:
#training data
df_train=pd.read_csv("/content/drive/My Drive/minor/english_dataset/english_dataset_prepro.csv")
#drop rows having value NONE in task 2
df_train=df_train[df_train.task_2!="NONE"]

In [4]:
df_train.head()

,text_id,text,task_1,task_2,task_3
1,hasoc_en_2,politico remember clearly individual1 admitted...,HOF,HATE,TIN
7,hasoc_en_8,ados trendingnow blacklivesmatter justice fuck...,HOF,PRFN,TIN
11,hasoc_en_12,don’t know much take 45 compulsive liar trump3...,HOF,HATE,TIN
15,hasoc_en_16,good work icc keep going destroy whole fucking...,HOF,PRFN,TIN
23,hasoc_en_24,shameonicc 1 icc dhoni gloves vs 2 icc plannin...,HOF,HATE,TIN


In [5]:
Column_Sequence=np.array(df_train["text"],dtype="str") #storing value  of tweets or text in numpy array of strings

In [6]:
Tokenizer=RegexpTokenizer(r"\w+[a-z]+") #create a reg exp tokenizer object for text to make a list of strings and lowercase text (earlier also removed stopwords but that has been moved to preprocessing code)

In [7]:
fresh=[]
for ix in Column_Sequence:
    sen=ix.lower()                        #lower case
    sen=Tokenizer.tokenize(sen)           #tokenize the sentences.
    wordss=[(Word) for Word in sen]
    sentance=" ".join(wordss)
    fresh.append(sentance)                #append all in one.

In [8]:
tokenizer = keras_text.Tokenizer(char_level = False,lower=True) #create a kerastokenizer object
t=tokenizer.fit_on_texts(list(fresh))                           #fit on training text
list_tokenized_train = tokenizer.texts_to_sequences(fresh)
X_t = keras_seq.pad_sequences(list_tokenized_train, maxlen=300,padding="post") #maxlen padding 100
word_index = tokenizer.word_index      #get a list of all tokens or words in the vocab

In [9]:
#extract word2vec vectors for words in vocab
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors
#load word2vec vectors from file.
word_vectors = KeyedVectors.load_word2vec_format('/content/drive/My Drive/minor/wordtovec/GoogleNews-vectors-negative300.bin.gz', binary=True)

EMBEDDING_DIM=300
vocabulary_size=len(word_index)+1
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
for word, i in word_index.items():
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector            #store the vector if found.
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)  #store a rndom vector if not found

del(word_vectors)  #del word2vec vectors to free memory

from keras.layers import Embedding
#create an embediing layer from word2vec vectors of our vocab
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [10]:
len(tokenizer.word_counts)

9499

In [11]:
X_train=X_t       #training text

In [12]:
X_train.shape

(2261, 300)

In [13]:
Y=df_train["task_2"]   #labels of classes with strings.

In [14]:
Y = Y.map({'PRFN':0, 'OFFN': 1,'HATE':2})  #map the class names to numbers.

In [15]:
# One-hot encode label
Y_train=np_utils.to_categorical(Y)

In [16]:
Y_train.shape

(2261, 3)

In [17]:
x_train, x_test, y_train, y_test = train_test_split(X_train,Y_train, test_size=0.10, random_state=42) #training and validation split of training data

In [18]:
y_train.shape

(2034, 3)

In [19]:
x_train.shape

(2034, 300)

In [20]:
max_len=300  #length of text sequence

In [21]:
x_train

array([[5201,  253,  592, ...,    0,    0,    0],
       [   6,   15, 1828, ...,    0,    0,    0],
       [1091, 6275, 1519, ...,    0,    0,    0],
       ...,
       [1652,  295,   30, ...,    0,    0,    0],
       [   2,  495,   30, ...,    0,    0,    0],
       [   4, 2114,  254, ...,    0,    0,    0]], dtype=int32)

In [22]:
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint
#CNN
sequence_input1 = Input(shape=(max_len,), dtype='int32')  #input layer
embedded_sequences1 = embedding_layer(sequence_input1)
l_cov1= Conv1D(128, 5, activation='relu')(embedded_sequences1)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(5)(l_cov2)
l_cov3 = Conv1D(128, 5, activation='relu')(l_pool2)
l_pool3 = MaxPooling1D(5)(l_cov3)  # global max pooling
l_flat1 = Flatten()(l_pool3)
l_dense1 = Dense(128, activation='relu')(l_flat1)
preds1 = Dense(3, activation='softmax')(l_dense1)      #output layer

model1 = Model(sequence_input1, preds1)
model1.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("Simplified convolutional neural network")
model1.summary()
cp1=ModelCheckpoint('model_cnn.hdf5',monitor='val_acc',verbose=1,save_best_only=True) #callback checkpoint


Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Simplified convolutional neural network
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 300, 300)          2850000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 296, 128)          192128    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 59, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 55, 128)           82048     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 11, 128)           0         
___

In [23]:
from keras.layers import Dense, Input, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, CuDNNGRU, Bidirectional, concatenate
from keras.models import Model
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint
#BiGRU
sequence_input2 = Input(shape=(max_len,), dtype='int32')   #input layer
embedded_sequences2 = embedding_layer(sequence_input2)
gru1=Bidirectional(CuDNNGRU(128, return_sequences=True))(embedded_sequences2)  #gpu optimized gru layer with a bi-directional layer
gru2= Bidirectional(CuDNNGRU(128, return_sequences=True))(gru1)
avg_pool = GlobalAveragePooling1D()(gru2)
max_pool = GlobalMaxPooling1D()(gru2)
conc = concatenate([avg_pool, max_pool])
l_dense2 = Dense(128, activation='relu')(conc)
preds2 = Dense(3, activation='softmax')(l_dense2)  #output layer

model2 = Model(sequence_input2, preds2)
model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

print("GRU")
model2.summary()
cp2=ModelCheckpoint('model_gru.hdf5',monitor='val_accuracy',verbose=1,save_best_only=True) #callback checkpoint

GRU
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 300)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 300, 300)     2850000     input_2[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 300, 256)     330240      embedding_1[1][0]                
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 300, 256)     296448      bidirectional_1[0][0]            
________________________________________________________________________________________

In [24]:
from keras.layers import Dense, Input, GlobalMaxPooling1D, CuDNNLSTM, GlobalAveragePooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM, Bidirectional, concatenate
from keras.models import Model
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint
#Bi-LSTM
sequence_input3 = Input(shape=(max_len,), dtype='int32') #input layer
embedded_sequences3 = embedding_layer(sequence_input3)
lstm1 = Bidirectional(CuDNNLSTM(128, return_sequences=True))(embedded_sequences3) #gpu optimized lstm layer with a bi-directional layer
lstm2= Bidirectional(CuDNNLSTM(128, return_sequences=True))(lstm1)
avg_pool = GlobalAveragePooling1D()(lstm2)
max_pool = GlobalMaxPooling1D()(lstm2)
conc = concatenate([avg_pool, max_pool])
l_dense3 = Dense(128, activation='relu')(conc)
preds3 = Dense(3, activation='softmax')(l_dense3) #output layer

model3 = Model(sequence_input3, preds3)
model3.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

print("LSTM")
model3.summary()
cp3=ModelCheckpoint('model_lstm.hdf5',monitor='val_accuracy',verbose=1,save_best_only=True) #callback checkpoint

LSTM
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 300)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 300, 300)     2850000     input_3[0][0]                    
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 300, 256)     440320      embedding_1[2][0]                
__________________________________________________________________________________________________
bidirectional_4 (Bidirectional) (None, 300, 256)     395264      bidirectional_3[0][0]            
_______________________________________________________________________________________

In [25]:
history1=model1.fit(x_train, y_train, validation_data=(x_test, y_test),epochs=15, batch_size=32,callbacks=[cp1]) #training cnn


Train on 2034 samples, validate on 227 samples
Epoch 1/15
2034/2034 [==============================] - 7s 4ms/step - loss: 0.9338 - acc: 0.5954 - val_loss: 0.8953 - val_acc: 0.6344

Epoch 00001: val_acc improved from -inf to 0.63436, saving model to model_cnn.hdf5
Epoch 2/15
2034/2034 [==============================] - 1s 310us/step - loss: 0.7292 - acc: 0.6908 - val_loss: 0.9572 - val_acc: 0.5903

Epoch 00002: val_acc did not improve from 0.63436
Epoch 3/15
2034/2034 [==============================] - 1s 286us/step - loss: 0.5087 - acc: 0.7989 - val_loss: 1.2531 - val_acc: 0.6564

Epoch 00003: val_acc improved from 0.63436 to 0.65639, saving model to model_cnn.hdf5
Epoch 4/15
2034/2034 [==============================] - 1s 255us/step - loss: 0.3433 - acc: 0.8673 - val_loss: 1.3056 - val_acc: 0.5066

Epoch 00004: val_acc did not improve from 0.65639
Epoch 5/15
2034/2034 [==============================] - 1s 249us/step - loss: 0.2200 - acc: 0.9331 - val_loss: 1.6057 - val_acc: 0.4493



In [26]:
history2=model2.fit(x_train, y_train, validation_data=(x_test, y_test),epochs=7, batch_size=32,callbacks=[cp2]) #training bi-gru

Train on 2034 samples, validate on 227 samples
Epoch 1/7
2034/2034 [==============================] - 6s 3ms/step - loss: 0.8740 - acc: 0.6288 - val_loss: 0.8592 - val_acc: 0.6211
Epoch 2/7
  64/2034 [..............................] - ETA: 4s - loss: 0.6915 - acc: 0.7344

/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:707: RuntimeWarning: Can save best model only with val_accuracy available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


2034/2034 [==============================] - 5s 3ms/step - loss: 0.6841 - acc: 0.7109 - val_loss: 0.8465 - val_acc: 0.6344
Epoch 3/7
2034/2034 [==============================] - 5s 2ms/step - loss: 0.5474 - acc: 0.7670 - val_loss: 0.9048 - val_acc: 0.6079
Epoch 4/7
2034/2034 [==============================] - 5s 3ms/step - loss: 0.4301 - acc: 0.8407 - val_loss: 1.0360 - val_acc: 0.6344
Epoch 5/7
2034/2034 [==============================] - 5s 3ms/step - loss: 0.2735 - acc: 0.9140 - val_loss: 1.2477 - val_acc: 0.6388
Epoch 6/7
2034/2034 [==============================] - 5s 3ms/step - loss: 0.2111 - acc: 0.9292 - val_loss: 1.1981 - val_acc: 0.6608
Epoch 7/7
2034/2034 [==============================] - 5s 3ms/step - loss: 0.1206 - acc: 0.9676 - val_loss: 1.3144 - val_acc: 0.6256


In [27]:
history3=model3.fit(x_train, y_train, validation_data=(x_test, y_test),epochs=7, batch_size=32,callbacks=[cp3])  #training bilstm

Train on 2034 samples, validate on 227 samples
Epoch 1/7
2034/2034 [==============================] - 7s 3ms/step - loss: 0.8769 - acc: 0.6126 - val_loss: 0.8698 - val_acc: 0.6300
Epoch 2/7
  64/2034 [..............................] - ETA: 5s - loss: 0.6409 - acc: 0.7500

/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:707: RuntimeWarning: Can save best model only with val_accuracy available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


2034/2034 [==============================] - 6s 3ms/step - loss: 0.7219 - acc: 0.6912 - val_loss: 0.8791 - val_acc: 0.6520
Epoch 3/7
2034/2034 [==============================] - 6s 3ms/step - loss: 0.6068 - acc: 0.7522 - val_loss: 0.9540 - val_acc: 0.5727
Epoch 4/7
2034/2034 [==============================] - 6s 3ms/step - loss: 0.5033 - acc: 0.7871 - val_loss: 0.9928 - val_acc: 0.5991
Epoch 5/7
2034/2034 [==============================] - 6s 3ms/step - loss: 0.3938 - acc: 0.8461 - val_loss: 1.1734 - val_acc: 0.6035
Epoch 6/7
2034/2034 [==============================] - 6s 3ms/step - loss: 0.2871 - acc: 0.8874 - val_loss: 1.2649 - val_acc: 0.6476
Epoch 7/7
2034/2034 [==============================] - 6s 3ms/step - loss: 0.2436 - acc: 0.9100 - val_loss: 1.3733 - val_acc: 0.6476


In [28]:
w1=model1.predict(x_test) #predict on val data cnn
w1=np.argmax(w1,axis=1)
w2=model2.predict(x_test) #predict on val data bi-gru
w2=np.argmax(w2,axis=1)
w3=model3.predict(x_test) #predict on val data bi-lstm
w3=np.argmax(w3,axis=1)

In [29]:
from sklearn.metrics import classification_report,confusion_matrix

In [30]:
#true labels for val data
Y_actual=[]
for ix in y_test:
    Y_actual.append(np.argmax(ix))

In [31]:
print("CNN")
print(classification_report(Y_actual,w1))
print("BI-GRU")
print(classification_report(Y_actual,w2))
print("BI-LSTM")
print(classification_report(Y_actual,w3))

CNN
              precision    recall  f1-score   support

           0       0.68      0.56      0.61        72
           1       0.27      0.15      0.20        39
           2       0.68      0.86      0.76       116

    accuracy                           0.64       227
   macro avg       0.55      0.52      0.52       227
weighted avg       0.61      0.64      0.62       227

BI-GRU
              precision    recall  f1-score   support

           0       0.68      0.69      0.69        72
           1       0.25      0.26      0.25        39
           2       0.72      0.71      0.71       116

    accuracy                           0.63       227
   macro avg       0.55      0.55      0.55       227
weighted avg       0.63      0.63      0.63       227

BI-LSTM
              precision    recall  f1-score   support

           0       0.67      0.68      0.68        72
           1       0.26      0.18      0.21        39
           2       0.72      0.78      0.75       116

 

In [32]:
data_test=pd.read_csv('/content/drive/My Drive/minor/english_dataset/hasoc2019_en_test-2919_prepro.csv') #testing data
data_test=data_test[data_test.task_2!="NONE"] #droping rows where task_2=NONE

In [33]:
data_test.head()

,text_id,text,task_1,task_2,task_3
4,hasoc_en_568,fuck go back dark ages cow ibnliverealtime rap...,HOF,PRFN,UNT
9,hasoc_en_527,jesus christ christian news illuminati changin...,HOF,HATE,TIN
10,hasoc_en_137,even true muslims supporting zakir knkmow prom...,HOF,PRFN,TIN
11,hasoc_en_606,kerala halal course sickular bastion serve por...,HOF,HATE,TIN
12,hasoc_en_246,look sweetie it’s simple jain ask veg food get...,HOF,PRFN,TIN


In [34]:
data_test.text=data_test.text.astype(str)
df_text=data_test['text']

In [35]:
Column_Sequence=np.array(data_test["text"],dtype="str")
fresh=[]
for ix in Column_Sequence:
    sen=ix.lower()
    sen=Tokenizer.tokenize(sen) #use the reg exp tokenizer object for text to make a list of strings and lowercase text (earlier also removed stopwords but that has been moved to preprocessing code)
    wordss=[(Word) for Word in sen]
    sentance=" ".join(wordss)
    fresh.append(sentance)

In [36]:
tokenizer = keras_text.Tokenizer(char_level = False,lower=True) #create a kerastokenizer object
t=tokenizer.fit_on_texts(list(fresh))
list_tokenized_train = tokenizer.texts_to_sequences(fresh)
X_test = keras_seq.pad_sequences(list_tokenized_train, maxlen=300,padding="post") #maxlen after padding 300

In [37]:
w_test1=model1.predict(X_test)     #predict class of testing data for cnn
w_test1=np.argmax(w_test1,axis=1)
w_test2=model2.predict(X_test)     #predict class of testing data for bi-gru
w_test2=np.argmax(w_test2,axis=1)
w_test3=model3.predict(X_test)     #predict class of testing data for bi-lstm
w_test3=np.argmax(w_test3,axis=1)

In [38]:
data_test['task_2'] = data_test['task_2'].map({'PRFN':0, 'OFFN': 1,'HATE':2}) #map label strings to numbers

In [39]:
y_test_actual=data_test['task_2']

In [40]:
print("CNN")
print(classification_report(y_test_actual,w_test1))
print("BI-GRU")
print(classification_report(y_test_actual,w_test2))
print("BI-LSTM")
print(classification_report(y_test_actual,w_test3))

CNN
              precision    recall  f1-score   support

           0       0.53      0.27      0.36        93
           1       0.22      0.08      0.12        71
           2       0.47      0.81      0.59       124

    accuracy                           0.45       288
   macro avg       0.41      0.39      0.36       288
weighted avg       0.43      0.45      0.40       288

BI-GRU
              precision    recall  f1-score   support

           0       0.45      0.33      0.38        93
           1       0.22      0.15      0.18        71
           2       0.51      0.69      0.59       124

    accuracy                           0.44       288
   macro avg       0.39      0.39      0.38       288
weighted avg       0.42      0.44      0.42       288

BI-LSTM
              precision    recall  f1-score   support

           0       0.51      0.44      0.47        93
           1       0.20      0.11      0.14        71
           2       0.50      0.68      0.58       124

 